In [1]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup as bs
import re

In [2]:
x = urllib.request.urlopen('https://www.indeed.com/viewjob?jk=09c378150b59ed91&tk=1d6kbudnu27er000&from=serp&vjs=3&advn=3432481575695357&adid=168292695&sjdu=EWPHJb1ZpAberNsuRbngZ9bczhETWqvjwR6coHtxZ0zjGIeQYX7jfYA5OcvOTyJsumyFQTKCf-JRlVUgR22EgpkmUdSqWQecAAh8grBl0AsyRgCTnK0wCTDMMViW5fn68oeMzArwA5pRSRhwd1IdSfLyW11X4lOQS8OtDdYVt876Qs1UKj56Ulh4wO3yRwXqn0Pd0SIgaio7wt5s-qjaenDlgu4ikmZEN-wgjmAFCc1CJxvOIjp3-1__stHYdweNgYexFP8pHXiksA_ReD_N750RB8uM0Lazr5Su6-J2mRg')
html_page = x.read()

In [3]:
a = bs(html_page, 'html.parser')

In [4]:
job_title = a.find("h3", {"class":"icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title"}).contents[0]
job_title

'Sales Associate'

In [5]:
conts = a.find("div",{"class":"jobsearch-JobComponent-description icl-u-xs-mt--md"}).contents[0]

In [6]:
conts

<div><div>CB2 Sales Associates are at the core of our customers’ experience. Friendly, outgoing and naturally driven, you’re a fan of everything we sell. That enthusiasm brings a level of excitement to our stores that makes shopping fun—and keeps customers coming back. Under the direction and mentorship of the Management Team, your day is dedicated to meeting and exceeding daily sales goals, becoming an expert on our products, answering customers’ questions and keeping the sales floor neat and ready to shop.</div><br/>
<ul><li>
Demonstrate and model selling skills, customer service, merchandising and teamwork.</li><li>
Support the management team in leadership, training, and promotion of teamwork throughout the sales floor and stockroom.</li><li>
Ensure full understanding of all products and have a continued awareness of the most current information available.</li><li>
Maintain an awareness of current advertising, promotions and catalogs as well as the status of merchandise.</li><li>
M